In [2]:
import numpy as np
import pandas as pd
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression




/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Set your project path 
project_path =  '/content/drive/My Drive/GL NLP Issue classifier capstone project/final_submission/'

In [5]:
tickets_df = pd.read_csv(project_path+'cleandata.csv')
print(tickets_df.shape)
tickets_df = tickets_df[tickets_df['CleanDescription'].notna()]
tickets_df.head()

(8500, 12)


,Unnamed: 0,Short description,Description,Caller,Assignment group,short_desc_language,desc_language,FullDescription,CleanDescription,CleanWordCount,WordCount,LabelEncodings
0,0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0,en,en,login issue -verified user details.(employee# ...,login issue verify user detail employee manage...,22,31,0
1,1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0,et,en,outlook \r\n\r\nreceived from: hmjdrvpb.komuay...,outlook team meet skype appear calendar somebo...,12,24,0
2,2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0,en,en,cant log in to vpn \r\n\r\nreceived from: eylq...,cant log vpn best,6,14,0
3,3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,en,en,unable to access hr_tool page unable to access...,unable access hr page,8,10,0
4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,no,no,skype error skype error,skype error,4,4,0


In [6]:

#This needs to go in data cleaning part of other notebook
#removin same word as that of caller from description
#removing caller name found on same row
tickets_df['CleanDescription'] = tickets_df['CleanDescription'].replace(to_replace=r'\b'+tickets_df['Caller']+r'\b', value='',regex=True)
#removing caller nae found in entire column
tickets_df['CleanDescription'] = tickets_df['CleanDescription'].replace(tickets_df['Caller'],'', regex = True)



In [7]:
#updating the cleaned word count
tickets_df['CleanWordCount']=[len(x.split()) for x in tickets_df['CleanDescription'].tolist()]

In [8]:
group_frequency = dict(tickets_df['Assignment group'].value_counts())
group_df = pd.DataFrame.from_dict(group_frequency, orient='index', columns=['frequency'])
group_df['percent representation'] = group_df['frequency'] / group_df['frequency'].sum()
group_df['cumulative percentage'] = group_df['percent representation'].cumsum(axis = 0)
group_df.head(35)

,frequency,percent representation,cumulative percentage
GRP_0,3975,0.469193,0.469193
GRP_8,661,0.078022,0.547214
GRP_24,289,0.034112,0.581327
GRP_12,257,0.030335,0.611662
GRP_9,252,0.029745,0.641407
GRP_2,241,0.028447,0.669854
GRP_19,215,0.025378,0.695231
GRP_3,200,0.023607,0.718839
GRP_6,184,0.021719,0.740557
GRP_13,145,0.017115,0.757672


In [9]:
group_frequency_filtered = dict(filter(lambda x: x[1] >= 50, group_frequency.items()))

In [10]:
print (group_frequency_filtered)

{'GRP_0': 3975, 'GRP_8': 661, 'GRP_24': 289, 'GRP_12': 257, 'GRP_9': 252, 'GRP_2': 241, 'GRP_19': 215, 'GRP_3': 200, 'GRP_6': 184, 'GRP_13': 145, 'GRP_10': 140, 'GRP_5': 129, 'GRP_14': 118, 'GRP_25': 116, 'GRP_33': 107, 'GRP_4': 100, 'GRP_29': 97, 'GRP_18': 88, 'GRP_16': 85, 'GRP_17': 81, 'GRP_7': 68, 'GRP_31': 67, 'GRP_34': 62, 'GRP_26': 56}


In [11]:
#Lets now filter the data frame and retain only the necessary groups(Filtered)
valid_grp_list = list(group_frequency_filtered)

In [12]:

clean_tickets_df = tickets_df.groupby(['LabelEncodings']).filter(lambda x: len(x) >= 50)

# Model Benchmarking

In [ ]:
def create_bag_of_words(X):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = 'english', ngram_range = (1,3), max_features = 10000) 
    train_data_features = vectorizer.fit_transform(X)
    train_data_features = train_data_features.toarray()
    
    # tfidf transform
    from sklearn.feature_extraction.text import TfidfTransformer
    tfidf = TfidfTransformer()
    tfidf_features = tfidf.fit_transform(train_data_features).toarray()

    # Get words in the vocabulary
    vocab = vectorizer.get_feature_names()

      
    return vectorizer, vocab, train_data_features, tfidf_features, tfidf

In [36]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def get_metrics(y_test, y_predicted):  
    # true positives / (true positives+false positives)
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                    average='weighted',zero_division=0,)             
    # true positives / (true positives + false negatives)
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted',zero_division=0)
    
    # harmonic mean of precision and recall
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted',zero_division=0)
    
    # true positives + true negatives/ total
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

In [13]:
compare_results = pd.DataFrame(columns=['Algorithm Name', 'Accuracy','Precision','Recall' ,'F1 Score'])

## Model Using Logistic Regression

In [14]:

X = clean_tickets_df['CleanDescription']
y = clean_tickets_df['Assignment group']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.25)

In [ ]:
vectorizer, vocab, train_data_features, tfidf_features, tfidf  =  create_bag_of_words(X_train)

In [ ]:
def train_logistic_regression(features, label):
    print ("Training the logistic regression model...")
    from sklearn.linear_model import LogisticRegression
    ml_model = LogisticRegression(C = 100,random_state = 0)
    ml_model.fit(features, label)
    print ('Finished')
    return ml_model

In [18]:
ml_model = train_logistic_regression(tfidf_features, y_train)

Training the logistic regression model...
Finished


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [19]:
test_data_features = vectorizer.transform(X_test)
# Convert to numpy array
test_data_features = test_data_features.toarray()
test_data_tfidf_features = tfidf.fit_transform(test_data_features)

# Convert to numpy array
test_data_tfidf_features = test_data_tfidf_features.toarray()

In [20]:
predicted_train_y = ml_model.predict(tfidf_features)
correctly_identified_train_y = predicted_train_y == y_train
accuracy = np.mean(correctly_identified_train_y) * 100
print ('Training Accuracy = %.0f%%' %accuracy)

Training Accuracy = 95%


In [21]:
predicted_y = ml_model.predict(test_data_tfidf_features)
correctly_identified_y = predicted_y == y_test
accuracy = np.mean(correctly_identified_y) * 100
print ('Test Accuracy = %.0f%%' %accuracy)

Test Accuracy = 70%


In [22]:
#confusion matrix

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predicted_y)

array([[928,   0,   2, ...,   2,   1,   1],
       [  0,   2,   0, ...,   0,   1,   0],
       [  9,   0,  15, ...,   0,   6,   1],
       ...,
       [  8,   0,   0, ...,   4,   0,   0],
       [  1,   1,   0, ...,   0, 145,   3],
       [  8,   0,   0, ...,   0,  36,  22]])

In [23]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_y, target_names=valid_grp_list))

              precision    recall  f1-score   support

       GRP_0       0.78      0.93      0.84      1003
       GRP_8       0.50      0.67      0.57         3
      GRP_24       0.83      0.41      0.55        37
      GRP_12       0.33      0.14      0.20         7
       GRP_9       0.71      0.58      0.64        67
       GRP_2       0.52      0.61      0.56        28
      GRP_19       0.61      0.42      0.50        33
       GRP_3       0.60      0.43      0.50         7
       GRP_6       0.42      0.36      0.38        14
      GRP_13       0.95      0.86      0.90        22
      GRP_10       0.56      0.50      0.53        18
       GRP_5       0.33      0.26      0.29        53
      GRP_14       0.62      0.44      0.52        63
      GRP_25       0.00      0.00      0.00        12
      GRP_33       0.50      0.15      0.24        13
       GRP_4       0.67      0.18      0.29        11
      GRP_29       0.71      0.79      0.75        63
      GRP_18       0.70    

In [24]:
accuracy, precision, recall, f1 = get_metrics(y_test, predicted_y)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.700, precision = 0.675, recall = 0.700, f1 = 0.670


In [25]:
compare_results = compare_results.append({'Algorithm Name': 'Logistic Regression','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

## Model Using Logistic Regression with Balance

In [26]:

tfidf_features.shape

(6097, 10000)

In [27]:
#trying to balance 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=777)
ros_tfidf_features, ros_y_train = ros.fit_sample(tfidf_features, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [28]:

ros_tfidf_features.shape

(104020, 10000)

In [29]:
ml_model_balanced = train_logistic_regression(ros_tfidf_features, ros_y_train)

Training the logistic regression model...
Finished


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [30]:
bal_predicted_train_y = ml_model.predict(ros_tfidf_features)
bal_correctly_identified_train_y = bal_predicted_train_y == ros_y_train
accuracy = np.mean(correctly_identified_train_y) * 100
print ('Balanced Data Training Accuracy = %.0f%%' %accuracy)

Balanced Data Training Accuracy = 95%


In [31]:
bal_y_pred = ml_model_balanced.predict(test_data_tfidf_features)
bal_y_valid = bal_y_pred == y_test
accuracy = np.mean(bal_y_valid) * 100
print ('Balanced data Test Accuracy = %.0f%%' %accuracy)

Balanced data Test Accuracy = 66%


In [32]:
print(classification_report(y_test, bal_y_pred, target_names=valid_grp_list))

              precision    recall  f1-score   support

       GRP_0       0.80      0.84      0.82      1003
       GRP_8       0.20      0.67      0.31         3
      GRP_24       0.82      0.49      0.61        37
      GRP_12       0.33      0.14      0.20         7
       GRP_9       0.61      0.58      0.60        67
       GRP_2       0.49      0.61      0.54        28
      GRP_19       0.54      0.42      0.47        33
       GRP_3       0.60      0.43      0.50         7
       GRP_6       0.44      0.50      0.47        14
      GRP_13       0.91      0.95      0.93        22
      GRP_10       0.41      0.39      0.40        18
       GRP_5       0.33      0.34      0.34        53
      GRP_14       0.53      0.44      0.48        63
      GRP_25       0.00      0.00      0.00        12
      GRP_33       0.50      0.15      0.24        13
       GRP_4       0.50      0.18      0.27        11
      GRP_29       0.71      0.78      0.74        63
      GRP_18       0.62    

In [33]:
accuracy, precision, recall, f1 = get_metrics(y_test, bal_y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.664, precision = 0.681, recall = 0.664, f1 = 0.660


In [34]:
compare_results = compare_results.append({'Algorithm Name': 'Logistic Regression with balanced','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

In [35]:
compare_results

,Algorithm Name,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.699951,0.674800,0.674800,0.670141
1,Logistic Regression with balanced,0.664043,0.680666,0.680666,0.659697


## Model using AdaBoostClassifier

In [37]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
abcl = AdaBoostClassifier(base_estimator=LogisticRegression(C = 100,random_state = 0))
abcl = abcl.fit(tfidf_features, y_train)
pred_AB =abcl.predict(test_data_tfidf_features)
acc_AB = accuracy_score(y_test, pred_AB)

print("AdaBoost accuracy: ", acc_AB)

AdaBoost accuracy:  0.5961633054599115


In [38]:
accuracy, precision, recall, f1 = get_metrics(y_test, pred_AB)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.596, precision = 0.542, recall = 0.596, f1 = 0.481


In [39]:
compare_results = compare_results.append({'Algorithm Name': 'AdaBoost with Logistic','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

## Model using Decision Tree

In [40]:
from sklearn.tree import DecisionTreeClassifier
dec_model = DecisionTreeClassifier(criterion = "entropy")
dec_model.fit(tfidf_features, y_train)
preds_test = dec_model.predict(test_data_tfidf_features)
preds_train = dec_model.predict(tfidf_features)

In [41]:
accuracy, precision, recall, f1 = get_metrics(y_test, preds_test)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.577, precision = 0.541, recall = 0.577, f1 = 0.548


In [42]:
compare_results = compare_results.append({'Algorithm Name': 'Decision Tree','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

## Model using SVM

In [43]:
from sklearn.svm import SVC
clf = SVC(kernel='linear').fit(tfidf_features, y_train)
y_pred = clf.predict(test_data_tfidf_features)
acc_score = accuracy_score(y_test, y_pred)
print("SVM-Linerar Score: ", acc_score)

SVM-Linerar Score:  0.692572552877521


In [44]:
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.693, precision = 0.657, recall = 0.693, f1 = 0.633


In [45]:
compare_results = compare_results.append({'Algorithm Name': 'SVM','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

## Model using Naives Bayes

In [46]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(tfidf_features, y_train)
y_pred = clf.predict(test_data_tfidf_features)
acc_score = accuracy_score(y_test, y_pred)
print("Naive Bayes Score: ", acc_score)

Naive Bayes Score:  0.6123954746679784


In [47]:
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.612, precision = 0.524, recall = 0.612, f1 = 0.498


In [48]:
compare_results = compare_results.append({'Algorithm Name': 'Naives Bayes','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

## Model using Bagging

In [49]:
from sklearn.ensemble import BaggingClassifier

bgcl = BaggingClassifier(n_estimators=150, max_samples= .7, bootstrap=True, oob_score=True, random_state=22)
bgcl = bgcl.fit(tfidf_features, y_train)
pred_BG =bgcl.predict(test_data_tfidf_features)
acc_BG = accuracy_score(y_test, pred_BG)
print(acc_BG)

0.6728971962616822


In [50]:
accuracy, precision, recall, f1 = get_metrics(y_test, pred_BG)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.673, precision = 0.637, recall = 0.673, f1 = 0.619


In [51]:
compare_results = compare_results.append({'Algorithm Name': 'Bagging','Accuracy':accuracy,'Precision':precision,'Recall':precision ,'F1 Score': f1}, ignore_index=True)

In [52]:
compare_results

,Algorithm Name,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.699951,0.674800,0.674800,0.670141
1,Logistic Regression with balanced,0.664043,0.680666,0.680666,0.659697
2,AdaBoost with Logistic,0.596163,0.542397,0.542397,0.480865
3,Decision Tree,0.576980,0.541159,0.541159,0.548395
4,SVM,0.692573,0.657079,0.657079,0.632620
5,Naives Bayes,0.612395,0.524112,0.524112,0.497564
6,Bagging,0.672897,0.637052,0.637052,0.618717


## Model Using Random Forest

In [53]:
from sklearn.ensemble import RandomForestClassifier
rfcl = RandomForestClassifier(random_state=100, n_jobs=-1, class_weight="balanced")
rfcl = rfcl.fit(tfidf_features, y_train)
pred_RF = rfcl.predict(test_data_tfidf_features)
acc_RF = accuracy_score(y_test, pred_RF)
print(acc_RF)

0.661091982292179


In [54]:
accuracy, precision, recall, f1 = get_metrics(y_test, pred_RF)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.661, precision = 0.647, recall = 0.661, f1 = 0.606


In [55]:
compare_results = compare_results.append({'Algorithm Name': 'Random Forest','Accuracy':accuracy,'Precision':precision,'Recall':recall, 'F1 Score': f1}, ignore_index=True)

## Model using KNN

In [56]:
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import zscore
NNH = KNeighborsClassifier(n_neighbors= 5 , weights = 'distance')
NNH.fit(tfidf_features, y_train)
predicted = NNH.predict(test_data_tfidf_features)
acc_KNN = accuracy_score(y_test, predicted)
print(acc_KNN)

0.6271519921298574


In [57]:
accuracy, precision, recall, f1 = get_metrics(y_test, predicted)
print("accuracy = %.3f, precision = %.3f, recall = %.3f, f1 = %.3f" % (accuracy, precision, recall, f1))

accuracy = 0.627, precision = 0.622, recall = 0.627, f1 = 0.542


In [58]:
compare_results = compare_results.append({'Algorithm Name': 'KNN','Accuracy':accuracy,'Precision':precision,'Recall':recall, 'F1 Score': f1}, ignore_index=True)

## Model using Deep Learning

In [19]:
MAX_SEQ_LENGTH=tickets_df['CleanWordCount'].max()
MAX_SEQ_LENGTH

431

In [20]:
from keras.preprocessing.text import Tokenizer
# The maximum number of words to be used
MAX_NB_WORDS = 50000
# Max number of words in ticket
MAX_SEQUENCE_LENGTH = MAX_SEQ_LENGTH
# This is fixed.
EMBEDDING_DIM = 300
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, oov_token='OOV',filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(clean_tickets_df['CleanDescription'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 9879 unique tokens.


In [21]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(clean_tickets_df['CleanDescription'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (7733, 431)


In [22]:
le = preprocessing.LabelEncoder()
clean_tickets_df["LabelEncodings"] = le.fit_transform(clean_tickets_df["Assignment group"])
Y = pd.get_dummies(clean_tickets_df["LabelEncodings"]).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (7733, 24)


In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(6186, 431) (6186, 24)
(1547, 431) (1547, 24)


In [24]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,SpatialDropout1D,LSTM,Flatten
from keras.callbacks import EarlyStopping 

## LSTM

In [65]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(35, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.25,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])



/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4878 samples, validate on 1626 samples
Epoch 1/10
4878/4878 [==============================] - 124s 25ms/step - loss: 2.3667 - accuracy: 0.4904 - val_loss: 1.9572 - val_accuracy: 0.5529
Epoch 2/10
4878/4878 [==============================] - 123s 25ms/step - loss: 1.7505 - accuracy: 0.5806 - val_loss: 1.7067 - val_accuracy: 0.5824
Epoch 3/10
4878/4878 [==============================] - 122s 25ms/step - loss: 1.4848 - accuracy: 0.6158 - val_loss: 1.5773 - val_accuracy: 0.6132
Epoch 4/10
4878/4878 [==============================] - 123s 25ms/step - loss: 1.2801 - accuracy: 0.6533 - val_loss: 1.4974 - val_accuracy: 0.6298
Epoch 5/10
4878/4878 [==============================] - 121s 25ms/step - loss: 1.0817 - accuracy: 0.7009 - val_loss: 1.4824 - val_accuracy: 0.6384
Epoch 6/10
4878/4878 [==============================] - 122s 25ms/step - loss: 0.9192 - accuracy: 0.7429 - val_loss: 1.4553 - val_accuracy: 0.6531
Epoch 7/10
4878/4878 [==============================] - 123s 25ms/step

In [66]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1626/1626 [==============================] - 10s 6ms/step
Test set
  Loss: 1.527
  Accuracy: 0.642


In [67]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [68]:
yhat_probs = model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(X_test, verbose=0)

In [69]:
testy_inverse = np.argmax(Y_test, axis=1).reshape(-1,1)

In [70]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy_inverse, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy_inverse, yhat_classes, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(testy_inverse, yhat_classes, average='weighted')
print('Recall: %f' % recall)
f1 = f1_score(testy_inverse, yhat_classes, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.642066
Precision: 0.624255
Recall: 0.642066
F1 score: 0.623407


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
compare_results = compare_results.append({'Algorithm Name': 'LSTM','Accuracy':accuracy,'Precision':precision,'Recall':recall, 'F1 Score': f1}, ignore_index=True)

## Bidirectional LSTM Model using Embedding

In [15]:
glove_file = '/content/drive/My Drive/GL NLP Issue classifier capstone project/Sanket_Notebook/glove.6B.zip'

In [16]:
#Extract Glove embedding zip file
from zipfile import ZipFile
with ZipFile(glove_file, 'r') as z:
  z.extractall()

In [17]:
EMBEDDING_FILE = './glove.6B.200d.txt'

embeddings = {}
for o in open(EMBEDDING_FILE):
    word = o.split(" ")[0]
    # print(word)
    embd = o.split(" ")[1:]
    embd = np.asarray(embd, dtype='float32')
    # print(embd)
    embeddings[word] = embd

Create a weight matrix for words in training docs

In [25]:
# get vocab size 

num_words = len(tokenizer.word_index) + 1
print(num_words)

9880


In [26]:
embedding_matrix = np.zeros((num_words, 200))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

In [29]:
# Create and Compile  Model
# Fit  model with a batch size of 100 and validation_split = 0.2. and state the validation accuracy
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalAveragePooling1D,TimeDistributed
from tensorflow.keras.models import Model, Sequential

### Embedding layer for hint 
## model.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
### Bidirectional LSTM layer for hint 
## model.add(Bidirectional(LSTM(128, return_sequences = True)))
import tensorflow as tf
# Setting the model parameters
embedding_size = 200


bimodel = Sequential()
bimodel.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
bimodel.add(Bidirectional(LSTM(128, return_sequences = True)))
bimodel.add(Dropout(0.5))
bimodel.add(TimeDistributed(Dense(100,activation='relu')))
bimodel.add(GlobalAveragePooling1D())
bimodel.add(Dense(24, activation='softmax'))


bimodel.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
#bimodel.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
bimodel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         1976000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 256)         336896    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 100)         25700     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                2424      
Total params: 2,341,020
Trainable params: 2,341,020
Non-trainable params: 0
____________________________________________

In [31]:
# Training the model
from tensorflow.keras.callbacks import EarlyStopping
epochs = 10
batch_size = 100
history = bimodel.fit(X_train, Y_train,epochs=epochs,batch_size = batch_size,validation_split=0.25,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.01)])

Epoch 1/10
47/47 [==============================] - 175s 4s/step - loss: 2.1571 - accuracy: 0.5087 - val_loss: 2.1106 - val_accuracy: 0.5139
Epoch 2/10
47/47 [==============================] - 178s 4s/step - loss: 2.1012 - accuracy: 0.5072 - val_loss: 2.0723 - val_accuracy: 0.5145
Epoch 3/10
47/47 [==============================] - 175s 4s/step - loss: 2.0025 - accuracy: 0.5258 - val_loss: 1.8028 - val_accuracy: 0.5902
Epoch 4/10
47/47 [==============================] - 177s 4s/step - loss: 1.6903 - accuracy: 0.5919 - val_loss: 1.6307 - val_accuracy: 0.5979
Epoch 5/10
47/47 [==============================] - 173s 4s/step - loss: 1.5242 - accuracy: 0.6064 - val_loss: 1.5906 - val_accuracy: 0.5921
Epoch 6/10
47/47 [==============================] - 173s 4s/step - loss: 1.4089 - accuracy: 0.6185 - val_loss: 1.5703 - val_accuracy: 0.5947
Epoch 7/10
47/47 [==============================] - 175s 4s/step - loss: 1.3129 - accuracy: 0.6439 - val_loss: 1.5795 - val_accuracy: 0.6160
Epoch 8/10
47

In [32]:
accr = bimodel.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

49/49 [==============================] - 19s 394ms/step - loss: 1.6470 - accuracy: 0.6186
Test set
  Loss: 1.647
  Accuracy: 0.619


In [33]:
yhat_probs = bimodel.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = bimodel.predict_classes(X_test, verbose=0)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [34]:
testy_inverse = np.argmax(Y_test, axis=1).reshape(-1,1)

In [37]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy_inverse, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy_inverse, yhat_classes, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(testy_inverse, yhat_classes, average='weighted')
print('Recall: %f' % recall)
f1 = f1_score(testy_inverse, yhat_classes, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.618617
Precision: 0.502857
Recall: 0.618617
F1 score: 0.548816


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
compare_results = compare_results.append({'Algorithm Name': 'LSTM- Bidirection with Time distributed','Accuracy':accuracy,'Precision':precision,'Recall':recall, 'F1 Score': f1}, ignore_index=True)

## Bi-directional LSTM without TimeDistributed



In [47]:

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional,TimeDistributed,GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential

import tensorflow as tf
# Setting the model parameters
embedding_size = 200

bimodelnotT = Sequential()
bimodelnotT.add(Embedding(num_words, embedding_size, weights = [embedding_matrix]))
bimodelnotT.add(Bidirectional(LSTM(64, return_sequences = True,dropout=0.1, recurrent_dropout=0.2)))
bimodelnotT.add(GlobalMaxPool1D())
#add one dense layer with 64
bimodelnotT.add(Dense(Y.shape[1], activation='softmax'))
bimodelnotT.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
#bimodel.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['sparse_categorical_accuracy'])
bimodelnotT.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, None, 200)         1976000   
_________________________________________________________________
bidirectional_9 (Bidirection (None, None, 128)         135680    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 24)                3096      
Total params: 2,114,776
Trainable params: 2,114,776
Non-trainable params: 0
_________________________________________________________________


In [48]:
from tensorflow.keras.callbacks import EarlyStopping
epochs = 15
batch_size = 100
history = bimodelnotT.fit(X_train, Y_train,
                          epochs=epochs,
                          batch_size = batch_size,
                          validation_split=0.25,
                          callbacks=[EarlyStopping(monitor='val_loss', patience=2, min_delta=0.01)])

Epoch 1/15
47/47 [==============================] - 151s 3s/step - loss: 2.2126 - accuracy: 0.4971 - val_loss: 1.7885 - val_accuracy: 0.5798
Epoch 2/15
47/47 [==============================] - 151s 3s/step - loss: 1.6413 - accuracy: 0.5874 - val_loss: 1.4872 - val_accuracy: 0.6050
Epoch 3/15
47/47 [==============================] - 152s 3s/step - loss: 1.3723 - accuracy: 0.6353 - val_loss: 1.3007 - val_accuracy: 0.6516
Epoch 4/15
47/47 [==============================] - 151s 3s/step - loss: 1.1686 - accuracy: 0.6795 - val_loss: 1.1734 - val_accuracy: 0.6703
Epoch 5/15
47/47 [==============================] - 151s 3s/step - loss: 0.9924 - accuracy: 0.7185 - val_loss: 1.0845 - val_accuracy: 0.6942
Epoch 6/15
47/47 [==============================] - 151s 3s/step - loss: 0.8454 - accuracy: 0.7646 - val_loss: 1.0564 - val_accuracy: 0.6981
Epoch 7/15
47/47 [==============================] - 151s 3s/step - loss: 0.7247 - accuracy: 0.8075 - val_loss: 1.0429 - val_accuracy: 0.7046
Epoch 8/15
47

In [49]:
yhat_probs = bimodelnotT.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = bimodelnotT.predict_classes(X_test, verbose=0)

In [50]:
testy_inverse = np.argmax(Y_test, axis=1).reshape(-1,1)

In [51]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy_inverse, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy_inverse, yhat_classes, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(testy_inverse, yhat_classes, average='weighted')
print('Recall: %f' % recall)
f1 = f1_score(testy_inverse, yhat_classes, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.727214
Precision: 0.699726
Recall: 0.727214
F1 score: 0.698205


In [52]:
compare_results = compare_results.append({'Algorithm Name': 'LSTM- Bidirection without Time distributed','Accuracy':accuracy,'Precision':precision,'Recall':recall, 'F1 Score': f1}, ignore_index=True)

In [54]:
compare_results.sort_values(by='Accuracy', ascending=False)

,Algorithm Name,Accuracy,Precision,Recall,F1 Score
1,LSTM- Bidirection without Time distributed,0.727214,0.699726,0.727214,0.698205
0,LSTM- Bidirection with Time distributed,0.618617,0.502857,0.618617,0.548816


In [57]:

print(classification_report(testy_inverse, yhat_classes))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       820
           1       0.62      0.26      0.37        19
           2       0.70      0.59      0.64        54
           3       0.60      0.52      0.56        29
           4       0.27      0.17      0.21        18
           5       0.38      0.50      0.43        10
           6       0.90      0.95      0.93        20
           7       0.50      0.42      0.46        19
           8       0.34      0.26      0.30        42
           9       0.66      0.45      0.54        55
          10       0.75      0.94      0.83        51
          11       0.38      0.38      0.38        16
          12       0.33      0.14      0.20         7
          13       0.50      0.33      0.40        15
          14       0.22      0.15      0.18        39
          15       0.00      0.00      0.00        15
          16       0.28      0.22      0.24        23
          17       1.00    